In [1]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import pandas as pd
import json
load_dotenv()

True

In [2]:
# Important variables
mixtral_repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"


mixtral_llm = HuggingFaceEndpoint(
    repo_id=mixtral_repo_id,
    temperature=0.3
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Spectra\.cache\huggingface\token
Login successful


In [3]:
# Load data
test_data = pd.read_csv("data/test.csv")
submission_data = pd.read_csv("data/sample_submission.csv")

In [4]:
mistral_prompt = """

Given the following set of (original passage, rewritten passage) pairs, please try to infer the prompt that was used to rewrite each original passage into the corresponding exact rewritten passage. Return ONLY the prompt used.

Here are some examples:
 - original: The old oak tree stood tall and majestic, its branches reaching skyward like outstretched arms. Its weathered bark told the story of generations passed, a silent witness to the ebb and flow of life in the woodland. A serene haven for birds and squirrels, the tree offered shelter and sustenance.
   rewritten: The old oak tree stood tall and majestic, its branches reaching skyward like outstretched arms. Its weathered bark told the story of generations passed, a silent witness to the ebb and flow of life in the woodland. A serene haven for birds and squirrels, the tree offered shelter and sustenance. Its leaves danced in the wind, whispering secrets to the forest creatures below.
   prompt: Reconstruct the passage from a third-person omniscient point of view.

 - original: Waves crashed against the rocky shoreline, their rhythmic roar a soothing symphony. The salty tang of the air filled my lungs as I gazed out at the vast, blue expanse. Seagulls soared overhead, their cries mingling with the gentle lapping of the tide. This moment of tranquility was a balm for the soul.
   rewritten:Waves crashed against the rocky shore, their roar a soothing symphony. The salty tang of the air filled my lungs as I gazed at the vast, blue expanse. Seagulls soared overhead, their cries echoing the gentle lapping of the tide. This moment of tranquility was a balm for the soul.
   prompt: Revise the passage using more concise and compact language, emphasizing streamlining the writing.

 - original: The city streets bustled with energy, a cacophony of sounds and sights. Skyscrapers reached for the heavens, their glass facades reflecting the ever-changing dance of light and shadow. Amidst the chaos, pockets of green oases offered respite, inviting weary travelers to pause and reconnect with the natural world.
   rewritten: The urban streets reverberate with a symphony of activity, a cacophony of sounds and sights. Skyscrapers ascend to celestial heights, their mirrored glass facades mirroring the ephemeral dance of light and shadow. Amidst the tumult, verdant enclaves afford respite, inviting weary travelers to pause and commune with the verdant tapestry of nature.
   prompt: Rework the passage in a more formal or academic tone, incorporating more complex sentence structures and elevated vocabulary.

Here are the messages:
original: {original_message}
rewritten: {rewritten_message}

Return ONLY the prompt in JSON format:
{{prompt:}}
"""

prompt_template = PromptTemplate.from_template(mistral_prompt)


In [ ]:

original_texts = test_data["original_text"].values
rewritten_texts = test_data["rewritten_text"].values

reconstructed_prompts = []
for original, rewritten in zip(original_texts, rewritten_texts):
    test_prompt = prompt_template.format(original_message=original,
                       rewritten_message=rewritten)

    response = mixtral_llm.invoke(test_prompt)
    response = response.replace("\n", "")
    try:
        dict_response = json.loads(response)
        temp_prompt = dict_response.get("prompt")
        reconstructed_prompts.append(temp_prompt)
    except json.JSONDecodeError:
        temp_prompt = response.split(":")[1]
        reconstructed_prompts.append(temp_prompt.strip())

test_data["rewrite_prompt"] = reconstructed_prompts
    
submission_data = test_data[["id", "rewrite_prompt"]]
    